## Simple linear regression

Watch the 9-minute video below for a visual explanation of simple linear regression as a line-fitting problem.

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/PaFPbb66DxQ?start=24" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe> 

[Explaining Linear Regression, by StatQuest](https://www.youtube.com/embed/PaFPbb66DxQ?start=24)
```


Then study the following sections to learn more about simple linear regression with examples in the textbook.

### Install libraries

If you are using Google Colab, you can skip this section. If you are using your own computer, you will need to install the following libraries unless they are already installed:

In [ ]:
!pip install statsmodels seaborn

### Import libraries and load data

Get ready by importing the APIs needed from respective libraries.
<!-- Firstly, we will import the required libraries and load the `Advertising` dataset. -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.formula.api import ols

%matplotlib inline

Load the [Advertising dataset](https://github.com/pykale/transparentML/blob/main/data/Advertising.csv) and display the column names, counts and data types.

<!-- Datasets available on https://www.statlearning.com/resources-first-edition -->

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Advertising.csv"

advertising_df = pd.read_csv(data_url, header=0, index_col=0)
advertising_df.info()

Display the first 5 rows for inspection. You can also click [Advertising dataset](https://github.com/pykale/transparentML/blob/main/data/Advertising.csv) to inspect the full data on GitHub, since this dataset is small. It is a good habit to inspect the data before using it.

In [ ]:
advertising_df.head()

### Linear relationship modelling for regression

Simple linear regression assumes that there is an approximately _linear_ relationship between a quantitative response  (output) $y$ and a single predictor (input) $x$. Mathematically, the linear relationship can be expressed as

```{math}
:label: y_approx_x
y \approx \beta_0 + \beta_1 x
```

where $\beta_0$ and $\beta_1$ are two unknown constants that represent the *bias* and *weight* of this linear model, which are also known as *intercept* and *slope*, respectively. Together, $\beta_0$ and $\beta_1$ are called the *model coefficients*, or *parameters*. We can describe the linear relationship as *regressing* $y$ onto $x$. For example, $x$ may represent `TV` advertising budget (in thousands of dollars) and $y$ may represent `sales` (in thousands of dollars) in the [Advertising dataset](https://github.com/pykale/transparentML/blob/main/data/Advertising.csv). Then we can regress sales onto TV by fitting the model:

<!-- \begin{equation} -->
$$
\textrm{sales} \approx \beta_0 + \beta_1 \times \textrm{TV}
$$
<!-- \end{equation} -->



### Estimating the coefficients

The goal of simple linear regression is to estimate the unknown parameters $\beta_0$ and $\beta_1$ from the data. Let 

$$
(x_1, y_1), (x_2, y_2), \ldots, (x_N, y_N)
$$

be the $N$ observations in the dataset, and $\hat{\beta}_0$ and $\hat{\beta}_1$ denote the estimated values of $\beta_0$ and $\beta_1$, respectively. The estimated values of $\hat{\beta}_0$ and $\hat{\beta}_1$ can be obtained by minimising the [*residual sum of squares* (RSS)](https://en.wikipedia.org/wiki/Residual_sum_of_squares) between the observed response values $y_i$ and the predicted response values $\hat{y}_i$:


\begin{equation}
\text{RSS} = \sum_{i=1}^N (y_i - \hat{y}_i)^2 = \sum_{i=1}^N (y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i)^2,
\end{equation}


where $y_i$ is the $i\text{th}$ observation of the response $y$, $\hat{y}_i$ is the $i\text{th}$ observation of the predicted response $\hat{y}$, and $x_i$ is the $i\text{th}$ observation of the predictor $x$. The least squares estimates of $\beta_0$ and $\beta_1$ are given as a [closed-form solution](https://mathworld.wolfram.com/Closed-FormSolution.html) by

\begin{equation}
\hat{\beta}_1 = \frac{\sum_{i=1}^N (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^N (x_i - \bar{x})^2},
\end{equation}

\begin{equation}
\hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x}.
\end{equation}

where $\bar{x}$ and $\bar{y}$ are the sample means of $x$ and $y$ respectively. The least squares estimates of $\beta_0$ and $\beta_1$ are obtained by minimising the RSS. The least squares line is given by


\begin{equation}
\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x.
\end{equation}

The least squares line, also known as the *regression line*, is the straight line that minimises the sum of squared residuals.

Run the code below to fit `sales` onto `TV` for the `Advertising` dataset using the API [`seaborn.regplot`](https://seaborn.pydata.org/generated/seaborn.regplot.html) from the library `seaborn`, which plots data and a linear regression model fit. You are encouraged to click the hyperlink to the API to learn more about its usage. The data are shown as red dots and the regression line is shown in blue.

In [ ]:
sns.regplot(
    x=advertising_df.TV,
    y=advertising_df.Sales,
    order=1,
    x_ci="ci",
    scatter_kws={"color": "r", "s": 9},
)
plt.xlim(-10, 310)
plt.ylim(ymin=0)
plt.show()

You should see a figure generated. This figure shows the least squares fit of `sales` onto `TV` for the `Advertising` dataset. The objective is to minimise the sum of squared residuals, which is the sum of the squared vertical distances, i.e. the _errors_, between the data points (red dots) and the least squares line (blue line). 

<!-- **Example: fitting a linear regression model and visualising regression coefficients using `scikit-learn`** -->
### Example: model fitting and visualisation using `scikit-learn`

Now we use the [`LinearRegression` class in `scikit-learn`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) to fit a linear regression model. The [`.fit()` method](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.fit) takes two arguments, the first is the predictor variable and the second is the response variable. The `.fit()` method returns an object that contains the estimated coefficients. The `.intercept_` and `.coef_` attributes of the fitted model can be used to obtain the estimated intercept ($\beta_0$) and slope ($\beta_1$) of the regression line. 

<!-- 
Note that the text in the book describes the coefficients based on unnormalised data, whereas the plot shows the model based on normalised data. The latter is visually more appealing for explaining the concept of a minimum RSS. I think that, in order not to confuse the reader, the values on the axis of the `beta_0` coefficients have been changed to correspond with the text. The axes on the plots below are unaltered. -->

Firstly, fit a linear regression model. Before fitting, we centre the data by subtracting the mean of each variable from each observation via a [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). This is a common practice in machine learning.

In [ ]:
# Regression coefficients (Ordinary Least Squares)
regr = LinearRegression()

scale = StandardScaler(with_mean=True, with_std=False)
X = scale.fit_transform(advertising_df.TV.values.reshape(-1, 1))
y = advertising_df.Sales

regr.fit(X, y)
print("Mean removed from input features: ", scale.mean_)
print("Regression model intercept (bias): ", regr.intercept_)
print("Regression model slop (weight): ", regr.coef_)

Check the input data.

In [ ]:
X.shape

Create grid coordinates for plotting and compute the minimum of RSS.

In [ ]:
beta_0 = np.linspace(regr.intercept_ - 2, regr.intercept_ + 2, 50)
beta_1 = np.linspace(regr.coef_ - 0.02, regr.coef_ + 0.02, 50)
xx, yy = np.meshgrid(beta_0, beta_1, indexing="xy")
Z = np.zeros((beta_0.size, beta_1.size))

# Calculate Z-values (RSS) based on grid of coefficients
for (i, j), v in np.ndenumerate(Z):
    Z[i, j] = ((y - (xx[i, j] + X.ravel() * yy[i, j])) ** 2).sum() / 1000

# minimised RSS
min_RSS = r"$\beta_0$, $\beta_1$ for minimised RSS"
min_rss = (
    np.sum((regr.intercept_ + regr.coef_ * X - y.values.reshape(-1, 1)) ** 2) / 1000
)
min_rss

Plot the RSS with corresponding $\beta_0$ and $\beta_1$ in 2D and 3D.

In [ ]:
fig = plt.figure(figsize=(15, 6))
fig.suptitle("RSS - Regression coefficients", fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection="3d")

# Left plot
CS = ax1.contour(xx, yy, Z, cmap=plt.cm.Set1, levels=[2.15, 2.2, 2.3, 2.5, 3])
ax1.scatter(regr.intercept_, regr.coef_[0], c="r", label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt="%1.1f")

# Right plot
ax2.plot_surface(xx, yy, Z, rstride=3, cstride=3, alpha=0.3)
ax2.contour(
    xx,
    yy,
    Z,
    zdir="z",
    offset=Z.min(),
    cmap=plt.cm.Set1,
    alpha=0.4,
    levels=[2.15, 2.2, 2.3, 2.5, 3],
)
ax2.scatter3D(regr.intercept_, regr.coef_[0], min_rss, c="r", label=min_RSS)
ax2.set_zlabel("RSS")
ax2.set_zlim(Z.min(), Z.max())
ax2.set_ylim(0.02, 0.07)

# settings common to both plots
for ax in fig.axes:
    ax.set_xlabel(r"$\beta_0$", fontsize=17)
    ax.set_ylabel(r"$\beta_1$", fontsize=17)
    ax.set_yticks([0.03, 0.04, 0.05, 0.06])
    ax.legend()

### Example explanation of system transparency

Run the cell below for understanding the system transparency for linear regression models.

In [ ]:
sns.regplot(
    x=advertising_df.TV,
    y=advertising_df.Sales,
    order=1,
    x_ci="ci",
    scatter_kws={"color": "r", "s": 9},
)

x1 = 100
y1 = (regr.intercept_ + regr.coef_ * (x1 - scale.mean_))[0]
print("Predicted sales for TV = 100: ", y1)
plt.plot((x1, x1), (0, y1), "k--", lw=1)
plt.plot((-10, x1), (y1, y1), "k--", lw=1)

x_mean = scale.mean_[0]
y_ = (regr.intercept_ + regr.coef_ * (x_mean - scale.mean_))[0]
print("Predicted sales for TV = %s (mean): " % x_mean, y_)
plt.plot((x_mean, x_mean), (0, y_), "k--", lw=1)
plt.plot((-10, x_mean), (y_, y_), "k--", lw=1)

x2 = 200
y2 = (regr.intercept_ + regr.coef_ * (x2 - scale.mean_))[0]
print("Predicted sales for TV = 200: ", y2)
plt.plot((x2, x2), (0, y2), "k--", lw=1)
plt.plot((-10, x2), (y2, y2), "k--", lw=1)

plt.xlim(-10, 310)
plt.ylim(ymin=0)
plt.show()

In the above example, we learnt a linear regression model $f(x)$ with two parameters, $\beta_0$ and $\beta_1$, from the data, where
- $\beta_0 = 14.0225 $ is the `sales` when input value `TV` equals to its mean, i.e., 147.0425, and
- $\beta_1 = 0.0475 $ is the change of units in the `sales` when `TV` increases by 1 unit.

Using these two estimated parameters, we can examine the system logic of the simple linear regression model to reveal its system transparency.

```{admonition} System transparency
:class: important

- When `TV`$=200$, the predicted `sales` $f(200) \approx 16.54 = \beta_1 \times (200 - 147.04) + f(147.04)$, which can be derived from the linear regression equation: $f(x) = \beta_1 \times (x - \bar{x}) + \beta_0 = \beta_1 \times (x - \bar{x}) + f(\bar{x}) $. 

- To produce an estimated `sales` of $\hat{y}$, e.g. 18, we locate 18 on the vertical axis and then move horizontally to find the fitted line to find the corresponding `TV` value on the horizontal axis, i.e., ~230, which can be analytically obtained using the inverse function of the learnt linear regression model $f(x)$ $x = \frac{\hat{y} - \beta_0}{\beta_1} + \bar{x}$ (giving 230.78).

``` 

### Assessing model accuracy via $R^2$

The accuracy of the linear model is dependent on the variability of the response $y$ and the predictor $x$. The variability of $y$ is measured by the variance of $y$, denoted by $\sigma_y^2$. The variability of $x$ is measured by the variance of $x$, denoted by $\sigma_x^2$. The [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination), denoted by $R^2$, is defined as

\begin{equation}
R^2 = \frac{\text{TSS} - \text{RSS}}{\text{TSS}} = 1 - \frac{\text{RSS}}{\text{TSS}}
\end{equation}

where $\text{TSS} = \sum_{i=1}^N (y_i - \bar{y})^2$ *is the total sum of squares*. Dividing the RSS by the total number of training samples gives the *mean squared error* (MSE) of the model. The MSE is the average squared distance between the observed response values and the response values predicted by the model. The MSE is also known as the *mean squared prediction error* (MSPE). The MSE is defined as

\begin{equation}
\text{MSE} = \frac{1}{N} \text{RSS} = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2 = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{\beta}_0 - \hat{\beta}_1 x_i)^2.
\end{equation}

The coefficient of determination $R^2$ measures the proportion of the _total_ variance in the response $y$ that is explained by the linear model using the predictor $x$. $R^2$ is always between 0 and 1: it is 0 when the regression line does not fit the data at all, and it is 1 when the regression line perfectly fits the data. $R^2$ is also known as the *coefficient of multiple determination* and it is a measure of the goodness of fit of the linear model. 

Watch the 11-minute video below to learn more about $R^2$

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/2AQKmw14mHM?start=16" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe> 

[Explaining $R^2$, by StatQuest](https://www.youtube.com/embed/2AQKmw14mHM?start=16)
```

Run the following code to fit a simple linear regression model using `scikit-learn`.

In [ ]:
regr = LinearRegression()

X = advertising_df.TV.values.reshape(-1, 1)
y = advertising_df.Sales

regr.fit(X, y)
print(regr.intercept_)
print(regr.coef_)

Then evaluate the learnt model with $R^2$ via [`r2_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) (Table 3.1 & 3.2 of the text book).

In [ ]:
sales_pred = regr.predict(X)
print("R2 score:", r2_score(y, sales_pred))
print("Mean squared error: ", mean_squared_error(y, sales_pred))

From the result, we can see that the simple linear regression model explains about 61% of the variance of the response data around its mean, leaving about 39% (RSS) unexplained.

### Standard errors and residual standard error

*Advanced: advanced contents are optional.

The linear relationship between $x$ and $y$ can be written in an equation, rather than an approximation earlier in Equation {eq}`y_approx_x`, as

\begin{equation}
y = \beta_0 + \beta_1 x + \epsilon,
\end{equation}

where $\epsilon$ is a random error term that represents the difference (i.e. the error, unexplained part) between the observed response $y$ and the true response $\beta_0 + \beta_1 x$. The error term $\epsilon$ is assumed to be normally distributed with mean zero and constant variance $\sigma^2$. The coefficient estimates $\hat{\beta}_0$ and $\hat{\beta}_1$ are only estimates of the true coefficients $\beta_0$ and $\beta_1$. We can quantify the accuracy of the estimates by computing the [*standard error*](https://en.wikipedia.org/wiki/Standard_error) of the estimates. The following formulars can be used to compute the standard error associated with $\hat{\beta}_1$ and $\hat{\beta}_0$:

\begin{equation}
\text{SE}(\hat{\beta}_{1})^2 = \frac{\sigma^2}{\sum_{i=1}^N (x_i - \bar{x})^2}, \quad \text{SE}(\hat{\beta}_0)^2 = \sigma^2 \left[\frac{1}{N} + \frac{\bar{x}^2}{\sum_{i=1}^N (x_i - \bar{x})^2} \right],
\end{equation}

where $\sigma^2$ is an estimate of the variance of the error term $\epsilon= y - (\beta_0 + \beta_1 x) $, $\hat{y}_i$ is the $i\text{th}$ observation of the predicted response $\hat{y}$, and $x_i$ is the $i\text{th}$ observation of the predictor $x$, $\bar{x}$ and $\bar{y}$ are the sample means of $x$ and $y$ respectively. 

In general, $\sigma^2$ is unknown, so we need to estimate it from the *residual standard error* (RSE) below

\begin{equation}
\text{RSE} = \sqrt{\frac{1}{N-2}\text{RSS}} = \sqrt{\frac{1}{N-2} \sum_{i=1}^N (y_i - \hat{y}_i)^2}.
\end{equation}

RSE is also _another way to assess the quality of a linear regression fit_ (in terms of error). Due to the presence of the error term $\epsilon$, perfect prediction is not possible even if we know the true regression line (unless $\epsilon$ is zero, rarely the case in practice). The RSE is an estimate of the standard deviation of $\epsilon$. It is the average amount that the response will deviate from the true regression line, measuring the lack of fit of the model to the data. The smaller the RSE, the better the model fits the data. 

### Confidence intervals

The standard error indicates the average amount that an estimate differs from its actual value. Thus, the standard error of $\hat{\beta}_1$ is a measure of the average amount that $\hat{\beta}_1$ will deviate from the its true value $\beta_1$. The standard error of $\hat{\beta}_0$ is a measure of the average amount that $\hat{\beta}_0$ will deviate from the its true value $\beta_0$.

Standard errors can be used to compute [confidence intervals](https://en.wikipedia.org/wiki/Confidence_interval). A 95\% is defined as a range of values such that with 95 \% probability, the range (interval) will contain the true unknown value of the parameter. The range is defined in terms of lower and upper limits computed from the sample of data. For linear regression, the 95% confidence intervals for $\beta_1$ and $\beta_0$ approximately takes the form

\begin{equation}
\hat{\beta}_1 \pm 2 \times \text{SE}(\hat{\beta}_1),\:\: \hat{\beta}_0 \pm 2 \times \text{SE}(\hat{\beta}_0).
\end{equation}

The above can be interpreted as there is an approximately 95% chance that the interview $[\hat{\beta}_1 - 2 \times \text{SE}(\hat{\beta}_1), \hat{\beta}_1 + 2 \times \text{SE}(\hat{\beta}_1)] will contain the true value of $\beta_1$, and the interval $[\hat{\beta}_0 - 2 \times \text{SE}(\hat{\beta}_0), \hat{\beta}_0 + 2 \times \text{SE}(\hat{\beta}_0)]$ will contain the true value of $\beta_0$.

Run the following code to compute the statistics, including the confidence intervals, of the learnt model using `statesmodels` (page 67 & Table 3.1 & 3.2 of the textbook).

In [ ]:
est = ols("Sales ~ TV", advertising_df).fit()
est.summary().tables[1]

Note that the above did _NOT_ standardise the input by removing its mean so its intercept differs from the one in the previous section. The above includes some additional statistics. The last two columns are the 95% confidence intervals. The 5th column is the $p$-values of the coefficients, which are used to test the significance of the coefficients. The $p$-values are computed using the [$t$-test](https://en.wikipedia.org/wiki/Student%27s_t-test) for the [null hypothesis](https://en.wikipedia.org/wiki/Null_hypothesis) that the coefficient is zero.  The 4th column is the $t$ statistic value. The $p$-values are used to determine whether the coefficient is statistically significant. The smaller the $p$-value, the more likely it is that the coefficient is statistically significant. The $p$-values are also used to determine whether the model is statistically significant. The model is statistically significant if the $p$-value is less than the significance level (e.g. 0.05).

We can verify that the RSS computed via `statsmodels` is the same as that obtained via `scikit-learn`.

In [ ]:
# RSS with regression coefficients
(
    (advertising_df.Sales - (est.params[0] + est.params[1] * advertising_df.TV)) ** 2
).sum() / 1000

### Exercise

**1**. This question involves the use of simple linear regression on the **Auto** dataset from [Tabel 1.2](https://pykale.github.io/transparentML/01-intro/organisation.html#datasets-table).

**a**. Use the **LinearRegression()** function to perform a simple linear regression with **mpg** as the response and **horsepower** as the predictor. Use the **summary()** function to print the results. Comment on the output.

   For example:

   >i. Is there a relationship between the predictor and the response?

   >ii. How strong is the relationship between the predictor and the response?

   >iii. Is the relationship between the predictor and the response positive or negative?

In [ ]:
# Write your code below to answer the question

Compare your answer with the reference solution below

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

auto_df = pd.read_csv("https://github.com/pykale/transparentML/raw/main/data/Auto.csv")

# Remove missing values
auto_df = auto_df.drop(auto_df[auto_df.values == "?"].index)
auto_df = auto_df.reset_index()


# Convert quantitive datatypes to numerics
datatypes = {
    "quant": [
        "mpg",
        "cylinders",
        "displacement",
        "horsepower",
        "weight",
        "acceleration",
        "year",
    ],
    "qual": ["origin", "name"],
}

quants = auto_df[datatypes["quant"]].astype(np.float_)

auto_df = pd.concat([quants, auto_df[datatypes["qual"]]], axis=1)

# The statsmodels library provides a convenient means to get the
# same statistics

X = auto_df["horsepower"]
X = sm.add_constant(X)  # add bias constant
y = auto_df["mpg"]

results = sm.OLS(y, X).fit()
print(results.summary())


# i. Yes, the low P-value associated with the t-statistic for horsepower suggests so.
# ii. For a unit increase in horsepower, our model predicts mpg will decrease by -0.1578. So for example, increasing horsepower by 10 is expected to decrease efficiency by -1.578 mpg.
# iii. Negative

**b**. Plot the response and predictor. Use the **regplot()** function to display the least-squared regression line.

In [ ]:
# Write your code below to answer the question

Compare your answer with the reference solution below

In [ ]:
# Let's plot our predicted regression
import seaborn as sns

y_pred = results.predict(X)
df = pd.concat([auto_df["horsepower"], auto_df["mpg"]], axis=1)
ax = sns.scatterplot(x="horsepower", y="mpg", data=df)
ax.plot(auto_df["horsepower"], y_pred, color="red");

**c**. Produce the diagnostic plots of the least squares regression fit. Comment on any problems you see with the fit.

In [ ]:
# Write your code below to answer the question

Compare your answer with the reference solution below

In [ ]:
# Providing powerful residual plots for simple AND multivariate
# linear regresssion
# - bring your own predictions
# - underlying stats available as pandas dataframe
# - visualise linearity and outliers in multiple dimensions

import matplotlib.pyplot as plt
from scipy import stats


def lm_stats(X, y, y_pred):
    """LEVERAGE & STUDENTISED RESIDUALS
    - https://en.wikipedia.org/wiki/Studentized_residual#How_to_studentize
    """
    # Responses as np array vector
    try:
        y.shape[1] == 1
        # take first dimension as vector
        y = y.iloc[:, 0]
    except:
        pass
    y = np.array(y)

    # Residuals
    residuals = np.array(y - y_pred)

    # Hat matrix
    H = np.array(X @ np.linalg.inv(X.T @ X)) @ X.T

    # Leverage
    h_ii = H.diagonal()

    ## Externally studentised residual
    # In this case external studentisation is most appropriate
    # because we are looking for outliers.

    # Estimate variance (externalised)
    σi_est = []
    for i in range(X.shape[0]):
        # exclude ith observation from estimation of variance
        external_residuals = np.delete(residuals, i)
        σi_est += [
            np.sqrt(
                (1 / (X.shape[0] - X.shape[1] - 1))
                * np.sum(np.square(external_residuals))
            )
        ]
    σi_est = np.array(σi_est)

    # Externally studentised residuals
    t = residuals / σi_est * np.sqrt(1 - h_ii)

    # Return dataframe
    return pd.DataFrame(
        {
            "residual": residuals,
            "leverage": h_ii,
            "studentised_residual": t,
            "y_pred": y_pred,
        }
    )


def lm_plot(lm_stats_df):
    """Provides residual plots based on results from lm_stat()"""
    # Parse stats
    t = lm_stats_df["studentised_residual"]
    h_ii = lm_stats_df["leverage"]
    y_pred = lm_stats_df["y_pred"]

    # setup axis for grid
    plt.figure(1, figsize=(16, 18))

    # Studentised residual plot
    plt.subplot(321)
    ax = sns.regplot(x=y_pred, y=t, lowess=True)
    plt.xlabel("Fitted values")
    plt.ylabel("Studentised residuals")
    plt.title("Externally studentised residual plot", fontweight="bold")
    # Draw Hastie and Tibshirani's bounds for possible outliers
    ax.axhline(y=3, color="r", linestyle="dashed")
    ax.axhline(y=-3, color="r", linestyle="dashed")

    # Normal Q-Q plot
    plt.subplot(322)
    ax = stats.probplot(t, dist="norm", plot=plt)
    plt.ylabel("Studentised residuals")
    plt.title("Normal Q-Q", fontweight="bold")

    # Standardised residuals
    plt.subplot(323)
    ax = sns.regplot(x=y_pred, y=np.sqrt(np.abs(t)), lowess=True)
    plt.xlabel("Fitted values")
    plt.ylabel("√Standardized residuals")
    plt.title("Scale-Location", fontweight="bold")

    # Residuals vs Leverage plot
    plt.subplot(324)
    ax = sns.scatterplot(x=h_ii, y=t)
    plt.xlabel("Leverage")
    plt.ylabel("Studentised residuals")
    plt.title("Externally studentised residual vs Leverage", fontweight="bold")


X = pd.concat([auto_df["horsepower"]], axis=1)
# Create the Design Matrix by adding constant bias variable
intercept_const = pd.DataFrame({"intercept": np.ones(X.shape[0])})
X = np.array(pd.concat([intercept_const, X], axis=1))

y = auto_df["mpg"]

lm_plot(lm_stats(X, y, y_pred))

```{toggle}

The above residual plot grid shows the relationship between the horsepower predictor and the mpg response. There are several things to note:

* Non-linearity of the data: The top-left residual plot exhibits a discernable pattern, in this case u-shaped, that suggests our linear model is not providing a optimal fit to our data - the relationship is non-linear. A discernable pattern in this plot suggests that our model is failing to account for some of the reducible variance in the responses. There is still a discernable pattern in the bottom-left plot suggesting that a quadratic transform only improves the fit of our model slightly.

* Heteroscedasticity – Non-constant variance of error terms The top-left residual plot exhibits a conical shape. This suggests that there is some heteroscedasticity in our predictor. The standardised plot (bottom-left) also exhibits this characteristic suggesting that standardisation doesn't alleviate the issue – to address this we might consider fitting our model by weighted least squares.

* Outliers and leverage: the bottom-right residual vs leverage plot suggests that there are several potential outliers (points in top-right of axis) that could be having a strong effect (leverage) on our model. We should add more predictors to our model to clarify outliers.

* The top-right plot shows that our studentised residuals have a slightly non-normal distribution (TODO: ellaborate)
```

In [ ]:
def lm_residual_corr_plot(lm_stats_df):
    r = lm_stats_df["residual"]
    # Residuals correlation
    plt.figure(1, figsize=(16, 5))
    ax = sns.lineplot(x=list(range(r.shape[0])), y=r)
    plt.xlabel("Observation")
    plt.ylabel("Residual")
    plt.title("Correlation of error terms", fontweight="bold")


lm_residual_corr_plot(lm_stats(X, y, y_pred))

```{toggle}
**Correlation of error terms:** The Correlation of Error Terms plot shows errors against ordered observations in our dataset. We see a slight increase in error above the 300th observation suggesting some correlation effect. This could mean that our estimated standard errors underestimate the true standard errors. Our confidence and prediction intervals may be narrower than they should be.
```